
# 🗃️ Supermaket sales

In [0]:
%run ../_utils

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached Unidecode-1.3.8-py3-none-any.whl (235 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.



## dependency

In [0]:
!pip install opendatasets

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 10.5 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.12-py3-none-any.whl size=102975 sha256=d029fdd0a170139db179f728b986966afd5e3efac9b46ad04bd9ce9c07665ece
  Stored in directory: /root/.cache/pip/wheels/58/88/6f/7902bde01c79e67b25087a1d62e5b4ba59a9e8bd7676c3fb07
Successfully built kaggle
  Attempting uninstall: certifi
    Found existing installation: certifi 2022.12.7
    Not uninstalling certifi at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-7c9c2378-f2bb-42f3-adfc-ffe54fef615a
  

In [0]:
%run ../_create_database

DataFrame[]

DataFrame[]

DataFrame[]


## download dataset from kaggle

O dataset escolhido foi o [Supermarket sales](https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales)

Como é um dataset estático, não faz sentido adicionar upsert e tampouco streaming. Mas conforme o padrão dos dados (adicional de horario de venda), na vida real poderia fazer sentido pelo menos um minibatch para ingestão dos dados

In [0]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales", data_dir="/FileStore/datasets/") # foi para file:/

Dataset URL: https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales


100%|██████████| 35.9k/35.9k [00:00<00:00, 328kB/s]

In [0]:
%fs ls file:/FileStore/datasets/supermarket-sales

path,name,size,modificationTime
file:/FileStore/datasets/supermarket-sales/supermarket_sales - Sheet1.csv,supermarket_sales - Sheet1.csv,131528,1715630237365



o arquivo foi salvo em `file:/FileStore/datasets/supermarket-sales/supermarket_sales - Sheet1.csv``


## Carregando o arquivo .csv para spark dataframe

aqui o .od fez eu usar o workspace files. [mais detalhes aqui](https://docs.databricks.com/en/files/index.html)

In [0]:
df = (
    spark.read.format("csv")
    .options(header="true", inferSchema=True)
    .load("file:/FileStore/datasets/supermarket-sales/supermarket_sales - Sheet1.csv")
)


## 🔎 First exploratory data analysis (descriptive)

In [0]:
display(df.take(10))

Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,2024-05-13T13:08:00Z,Ewallet,522.83,4.761904762,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,2019-03-08,2024-05-13T10:29:00Z,Cash,76.4,4.761904762,3.82,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,2024-05-13T13:23:00Z,Credit card,324.31,4.761904762,16.2155,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,2019-01-27,2024-05-13T20:33:00Z,Ewallet,465.76,4.761904762,23.288,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,2024-05-13T10:37:00Z,Ewallet,604.17,4.761904762,30.2085,5.3
699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.39,7,29.8865,627.6165,2019-03-25,2024-05-13T18:30:00Z,Ewallet,597.73,4.761904762,29.8865,4.1
355-53-5943,A,Yangon,Member,Female,Electronic accessories,68.84,6,20.652,433.692,2019-02-25,2024-05-13T14:36:00Z,Ewallet,413.04,4.761904762,20.652,5.8
315-22-5665,C,Naypyitaw,Normal,Female,Home and lifestyle,73.56,10,36.78,772.38,2019-02-24,2024-05-13T11:38:00Z,Ewallet,735.6,4.761904762,36.78,8.0
665-32-9167,A,Yangon,Member,Female,Health and beauty,36.26,2,3.626,76.146,2019-01-10,2024-05-13T17:15:00Z,Credit card,72.52,4.761904762,3.626,7.2
692-92-5582,B,Mandalay,Member,Female,Food and beverages,54.84,3,8.226,172.746,2019-02-20,2024-05-13T13:27:00Z,Credit card,164.52,4.761904762,8.226,5.9


In [0]:
print(f"Total de linhas: {df.count()}")

Total de linhas: 1000


In [0]:
df.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)




O conjunto de dados é um dos registros históricos de vendas de uma empresa de supermercados, que foi registrado em 3 filiais diferentes ao longo de 3 meses. 

### schema
 - ``Invoice`` ID: Número de identificação da fatura de venda gerado pelo computador
 - ``Branch``: Filial do supermercado (3 filiais disponíveis identificadas por A, B e C).
 - ``City``: Localização dos supermercados
 - ``Customer`` type: Tipo de clientes, registrados como _Member_ para clientes que usam cartão de membro e Normal para aqueles sem cartão de membro.
 - ``Gender``: Tipo de gênero do cliente
 - ``Product`` line: Grupos de categorização geral de itens - Acessórios eletrônicos, Acessórios de moda, Alimentos e bebidas, Saúde e beleza, Casa e estilo de vida, Esportes e viagens
 - ``Unit price``: Preço de cada produto em USD ($)
 - ``Quantity``: Número de produtos comprados pelo cliente
 - ``Tax 5%``: Taxa de imposto de 5% para clientes que compraram
 - ``Total``: Preço total incluindo impostos
 - ``Date``: Data da compra (Registro disponível de janeiro de 2019 a março de 2019)
 - ``Time``: timestamp bugado, Hora da compra (das 10h às 21h)
 - ``Payment``: Método de pagamento usado pelo cliente na compra (3 métodos estão disponíveis - Dinheiro, Cartão de crédito e Ewallet)
 - ``cogs``: Custo dos bens vendidos
 - ``gross margin percentage``: Porcentagem de margem bruta
 - ``gross income``: Renda bruta
 - ``Rating``: Classificação de estratificação do cliente em sua experiência de compra geral (Em uma escala de 1 a 10)


## Saving data

salvando todos os .csv no dbf em arquivo parquet

Nossa staging poderia ser um S3 (aws) ou blob storage, mas vai ser diretamente no dbfs mesmo

In [0]:
file_name = "supermarket_sales"

In [0]:
save_dataframe(
    df,
    format_mode="parquet",
    target_location=f"dbfs:/FileStore/parquet/supermarket_sales/",
    table_name=file_name
)

[LOG] Saving supermarket_sales parquet on dbfs:/FileStore/parquet/supermarket_sales/... OK!


In [0]:
%fs ls '/FileStore/parquet/supermarket_sales/'

path,name,size,modificationTime
dbfs:/FileStore/parquet/supermarket_sales/_SUCCESS,_SUCCESS,0,1715630826000
dbfs:/FileStore/parquet/supermarket_sales/_committed_7481708366168257622,_committed_7481708366168257622,123,1715630826000
dbfs:/FileStore/parquet/supermarket_sales/_started_7481708366168257622,_started_7481708366168257622,0,1715630826000
dbfs:/FileStore/parquet/supermarket_sales/part-00000-tid-7481708366168257622-53dacee5-5446-42f1-9c5a-41a0d00de06f-23-1-c000.snappy.parquet,part-00000-tid-7481708366168257622-53dacee5-5446-42f1-9c5a-41a0d00de06f-23-1-c000.snappy.parquet,50430,1715630826000


In [0]:
dbutils.notebook.exit("OK")


#### test

In [0]:
display(spark.read.parquet("dbfs:/FileStore/parquet/supermarket_sales/").take(10))

Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,2024-05-13T13:08:00Z,Ewallet,522.83,4.761904762,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,2019-03-08,2024-05-13T10:29:00Z,Cash,76.4,4.761904762,3.82,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,2024-05-13T13:23:00Z,Credit card,324.31,4.761904762,16.2155,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,2019-01-27,2024-05-13T20:33:00Z,Ewallet,465.76,4.761904762,23.288,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,2024-05-13T10:37:00Z,Ewallet,604.17,4.761904762,30.2085,5.3
699-14-3026,C,Naypyitaw,Normal,Male,Electronic accessories,85.39,7,29.8865,627.6165,2019-03-25,2024-05-13T18:30:00Z,Ewallet,597.73,4.761904762,29.8865,4.1
355-53-5943,A,Yangon,Member,Female,Electronic accessories,68.84,6,20.652,433.692,2019-02-25,2024-05-13T14:36:00Z,Ewallet,413.04,4.761904762,20.652,5.8
315-22-5665,C,Naypyitaw,Normal,Female,Home and lifestyle,73.56,10,36.78,772.38,2019-02-24,2024-05-13T11:38:00Z,Ewallet,735.6,4.761904762,36.78,8.0
665-32-9167,A,Yangon,Member,Female,Health and beauty,36.26,2,3.626,76.146,2019-01-10,2024-05-13T17:15:00Z,Credit card,72.52,4.761904762,3.626,7.2
692-92-5582,B,Mandalay,Member,Female,Food and beverages,54.84,3,8.226,172.746,2019-02-20,2024-05-13T13:27:00Z,Credit card,164.52,4.761904762,8.226,5.9
